# Walmart Retail Sales Exploratory Analysis

This notebook provides an end-to-end exploratory data analysis (EDA) pipeline for the Walmart retail sales dataset used in this project.

**Objectives**

1. Load the raw dataset directly from the project structure.
2. Audit data quality and understand the distribution of key fields.
3. Engineer time-aware features that improve downstream forecasting models.
4. Generate exploratory visualizations to surface seasonal and store-level trends.
5. Persist an enriched feature set back into the project `processed/` directory.

> ℹ️ This notebook is designed to run from either the repository root or within the Walmart project folder. It assumes the virtual environment in `.venv/` is active so the dependencies declared in `requirements.txt` are available.

In [ ]:
from __future__ import annotations

import math
from pathlib import Path
from typing import Final

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

def resolve_project_dir(project_name: str = "retail_sales_walmart") -> Path:
    # Locate the Walmart project directory regardless of the launch location.
    # The search checks common entry points: repo root, project root, or notebooks folder.
    cwd = Path.cwd().resolve()

    candidate = cwd
    if (candidate / "projects" / project_name).exists():
        return candidate / "projects" / project_name

    if candidate.name == project_name and (candidate / "data").exists():
        return candidate

    if candidate.name == "notebooks" and (candidate.parent / "data").exists():
        return candidate.parent

    for parent in candidate.parents:
        if (parent / "projects" / project_name).exists():
            return parent / "projects" / project_name
        if parent.name == project_name and (parent / "data").exists():
            return parent

    raise FileNotFoundError(
        "Unable to locate project directory. Please run the notebook from the repo root or "
        "ensure the current working directory contains the retail_sales_walmart project."
    )

PROJECT_DIR: Final[Path] = resolve_project_dir()
DATA_DIR: Final[Path] = PROJECT_DIR / "data"
RAW_DATA_PATH: Final[Path] = DATA_DIR / "raw" / "Walmart.csv"
PROCESSED_DIR: Final[Path] = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project directory: {PROJECT_DIR}")
print(f"Using data source: {RAW_DATA_PATH.relative_to(PROJECT_DIR)}")
print(f"Processed outputs will be stored under: {PROCESSED_DIR.relative_to(PROJECT_DIR)}")

In [ ]:
df_raw = pd.read_csv(RAW_DATA_PATH)
df_raw.head()

## 1. Data quality checks

In [ ]:
row_count, col_count = df_raw.shape
print(f"Rows: {row_count:,} | Columns: {col_count}")

display(df_raw.describe(include='all').transpose())

missing_summary = (
    df_raw.isna()
    .sum()
    .rename("missing_count")
    .to_frame()
    .assign(missing_pct=lambda d: (d["missing_count"] / row_count) * 100)
)
display(missing_summary)

duplicates = df_raw.duplicated(subset=["Store", "Date"]).sum()
print(f"Duplicate [Store, Date] rows: {duplicates}")

## 2. Baseline feature engineering

In [ ]:
df = df_raw.copy()
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df.sort_values(["Store", "Date"], inplace=True)
df.reset_index(drop=True, inplace=True)

df["Holiday_Flag"] = df["Holiday_Flag"].astype(bool)
df["Week"] = df["Date"].dt.isocalendar().week.astype(int)
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Quarter"] = df["Date"].dt.quarter
df["DayOfYear"] = df["Date"].dt.dayofyear

# Seasonal encodings
df["Week_sin"] = np.sin(2 * math.pi * df["Week"] / 52)
df["Week_cos"] = np.cos(2 * math.pi * df["Week"] / 52)
df["Month_sin"] = np.sin(2 * math.pi * df["Month"] / 12)
df["Month_cos"] = np.cos(2 * math.pi * df["Month"] / 12)

# Climate conversions
df["Temperature_C"] = (df["Temperature"] - 32) * 5.0 / 9.0
df["Fuel_Price_Log"] = np.log1p(df["Fuel_Price"])

df.head()

In [ ]:
# Store-level aggregates
store_aggregates = (
    df.groupby("Store")["Weekly_Sales"]
    .agg(["mean", "median", "std", "max", "min"])
    .rename(columns=lambda c: f"Weekly_Sales_{c}")
)

df = df.merge(store_aggregates, on="Store", how="left")
display(store_aggregates.head())

In [ ]:
# Lag and rolling statistics per store
lag_features = [1, 2, 4, 13]  # weekly lags (1 week, 2 weeks, monthly, quarterly)
rolling_windows = [4, 8, 13]  # monthly, two-month, quarterly

for lag in lag_features:
    df[f"Weekly_Sales_lag{lag}"] = df.groupby("Store")["Weekly_Sales"].shift(lag)

for window in rolling_windows:
    df[f"Weekly_Sales_ma{window}"] = (
        df.groupby("Store")["Weekly_Sales"]
        .transform(lambda s: s.shift(1).rolling(window=window).mean())
    )
    df[f"Weekly_Sales_std{window}"] = (
        df.groupby("Store")["Weekly_Sales"]
        .transform(lambda s: s.shift(1).rolling(window=window).std())
    )

df.head(10)

## 3. Exploratory visualizations

In [ ]:
_ = sns.lineplot(data=df, x="Date", y="Weekly_Sales", hue="Store", legend=False)
plt.title("Weekly sales per store")
plt.ylabel("Weekly Sales (USD)")
plt.show()

In [ ]:
store_summary = (
    df.groupby("Store")["Weekly_Sales"]
    .agg(["mean", "median", "std", "max"])
    .sort_values("mean", ascending=False)
)
display(store_summary.head(10))

_ = sns.barplot(
    x=store_summary.index,
    y=store_summary["mean"],
    order=store_summary.index,
    palette="crest"
)
plt.title("Average weekly sales by store")
plt.xlabel("Store")
plt.ylabel("Average Weekly Sales (USD)")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
corr_features = [
    "Weekly_Sales",
    "Temperature",
    "Fuel_Price",
    "CPI",
    "Unemployment",
    "Week",
    "Month",
    "Quarter",
    "Weekly_Sales_lag1",
    "Weekly_Sales_ma4",
]

corr_matrix = df[corr_features].dropna().corr()
_ = sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation heatmap for key drivers")
plt.show()

## 4. Save engineered feature set

In [ ]:
df_features = df.copy()
df_features.sort_values(["Store", "Date"], inplace=True)

OUTPUT_PATH = PROCESSED_DIR / "walmart_features.parquet"
df_features.to_parquet(OUTPUT_PATH, index=False)
print(f"Saved engineered features to {OUTPUT_PATH.relative_to(PROJECT_DIR)}")
df_features.head()

## 5. Next steps

- Feed `walmart_features.parquet` into the Hydra training pipeline (see `src/ml_portfolio/conf/dataset/walmart.yaml`).
- Experiment with additional covariates (holiday calendars, promotional events) and re-run this notebook.
- Consider scaling features (e.g., standardization) in the training pipeline using the engineered columns created here.